# PySpark MLlib x Scikit-Learn

| Feature | scikit-learn | PySpark ML |
|---|---|---|
| **Scale** | single machine | Distributes processing |
| **Speed** | faster for smaller datasets | Slower for small datasets, but superior performance for massive data |
| **API/Usability** | Integrates well with classic Python ecosystem (Pandas, Matplotlib) | The API is tailored for distributed systems, with a different approach (e.g., using VectorAssembler for feature consolidation). |
| **Algorithms** | Offers a wider variety of algorithms and feature selection tools. | Includes core machine learning algorithms optimized for distributed processing|

# Algoritmos de Machine Learning Comuns
Dentro do temos algumas opções de modelos
Separei uma lista de modelos que funciona para Regressão e Classificação
- Generalized Linear Regression
    - Generalização da regressão linear ordinária. Podendo ser regressão linear, logistica poisson, ...
- Decision Tree
    - Árvores de Decisão (aprendizado supervisionado não paramétricos)

- Random Forest
    - Opera construindo uma infinidade de árvores de decisão

- Gradient Boosting
    - Conjunto de modelos de previsão fracos. Constrói o modelo em etapas, boosting, permitindo a otimização de uma função de perda diferenciável.

# Pipeline de Machine Learning (Classification)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, MinMaxScaler, Imputer
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator

spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/30 09:17:54 WARN Utils: Your hostname, MacBook-Air-de-Vitor.local, resolves to a loopback address: 127.0.0.1; using 192.168.3.49 instead (on interface en0)
26/01/30 09:17:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/30 09:17:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/30 09:17:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/30 09:17:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
df = spark.read.parquet("/Users/vho/alura/pyspark_course/aulas/olist.parquet")

train, test = df.randomSplit([0.7, 0.3], seed=42)

In [3]:
categorical_cols = ['order_status', 'payment_primary_method']
numeric_features = [
    'order_days_between',
    'order_count_item',
    'order_sum_price',
    'order_min_price',
    'order_max_price',
    'order_avg_price',
    'order_sum_freight_value',
    'order_min_freight_value',
    'order_max_freight_value',
    'order_avg_freight_value',
    'payment_count',
    'payment_min_value',
    'payment_max_value',
    'payment_avg_value',
    'payment_sum_value',
    'payment_max_installments',
    'payment_primary_value',
    'payment_primary_share'
]
assembler_inputs = numeric_features + [c + "_vec" for c in categorical_cols]

imputer=Imputer(inputCols=['order_days_between'], outputCols=['order_days_between']).setStrategy('median')
stringIndexer=StringIndexer(inputCols=categorical_cols, outputCols=[c + "_index" for c in categorical_cols])
oneHotEncoder=OneHotEncoder(inputCols=[c + "_index" for c in categorical_cols], outputCols=[c + "_vec" for c in categorical_cols])
vectorAssembler=VectorAssembler(inputCols=assembler_inputs, outputCol="features_raw")
minMaxScaler=MinMaxScaler(inputCol="features_raw", outputCol="features")

stages = [
    imputer,
    stringIndexer,
    oneHotEncoder,
    vectorAssembler,
    minMaxScaler
]

In [4]:
stages.append(GBTClassifier(featuresCol='features', labelCol='target', maxIter=10))

pipeline = Pipeline(stages=stages)

In [5]:
model = pipeline.fit(train)

26/01/30 09:17:58 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
predictions = model.transform(test)

In [7]:
predictions.select("rawPrediction", "probability", "prediction", "target").show(5, truncate=False)
# Raw Prediction: Output da arvore de decisão, simetrico
# Probability: Probabilidade de cada classe (sigmoid) sum=1
# Prediction: Classe prevista (corte 0.5)

+----------------------------------------+----------------------------------------+----------+------+
|rawPrediction                           |probability                             |prediction|target|
+----------------------------------------+----------------------------------------+----------+------+
|[-0.8289489528242735,0.8289489528242735]|[0.16004437995795773,0.8399556200420423]|1.0       |1     |
|[-0.8523026280645966,0.8523026280645966]|[0.15386474881524081,0.8461352511847592]|1.0       |1     |
|[-0.865401130581826,0.865401130581826]  |[0.15048499051308914,0.8495150094869108]|1.0       |1     |
|[-0.8542145655766816,0.8542145655766816]|[0.15336757569579207,0.8466324243042079]|1.0       |1     |
|[0.1489848405117163,-0.1489848405117163]|[0.5739461136055757,0.4260538863944243] |0.0       |1     |
+----------------------------------------+----------------------------------------+----------+------+
only showing top 5 rows


26/01/30 09:18:06 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [8]:
evaluator_binary = BinaryClassificationEvaluator(labelCol="target")
evaluator_multiclass = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction")

# Usando probability
print(f"Area Under ROC: {evaluator_binary.evaluate(predictions, {evaluator_binary.metricName: 'areaUnderROC'})}")
print(f"Area Under PR: {evaluator_binary.evaluate(predictions, {evaluator_binary.metricName: 'areaUnderPR'})}")

# Usando prediction
print(f"Accuracy: {evaluator_multiclass.evaluate(predictions, {evaluator_multiclass.metricName: 'accuracy'})}")
print(f"F1: {evaluator_multiclass.evaluate(predictions, {evaluator_multiclass.metricName: 'f1'})}")
print(f"Precision: {evaluator_multiclass.evaluate(predictions, {evaluator_multiclass.metricName: 'weightedPrecision'})}")
print(f"Recall: {evaluator_multiclass.evaluate(predictions, {evaluator_multiclass.metricName: 'weightedRecall'})}")

Area Under ROC: 0.6960817953946045
Area Under PR: 0.8734206204473458
Accuracy: 0.8167445455503636
F1: 0.7770910154424593
Precision: 0.7969131250592675
Recall: 0.8167445455503637


# Pipeline de Machine Learning (Regression)

Agora que já explicamos classificação, vamos para regressão. Dessa vez coloque o GBTRegressor no pipeline fazendo o fit e prediction na mesma celula.

In [9]:
train_reg = train.withColumn("review_score", F.col("review_score").cast("double"))
test_reg = test.withColumn("review_score", F.col("review_score").cast("double"))

stages = [
    imputer,
    stringIndexer,
    oneHotEncoder,
    vectorAssembler,
    minMaxScaler
]

gbt_reg = GBTRegressor(featuresCol='features', labelCol='review_score')
stages.append(gbt_reg)

pipeline_reg = Pipeline(stages=stages)

model_reg = pipeline_reg.fit(train_reg)
predictions_reg = model_reg.transform(test_reg)

In [10]:
evaluator_reg = RegressionEvaluator(labelCol="review_score", predictionCol="prediction")

rmse = evaluator_reg.evaluate(predictions_reg, {evaluator_reg.metricName: "rmse"})
r2 = evaluator_reg.evaluate(predictions_reg, {evaluator_reg.metricName: "r2"})

print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 1.1659161430976759
R2: 0.17153392787111132
